<a href="https://colab.research.google.com/github/vischia/pv_data_science_school/blob/master/1_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning School, ICNFP 2025 edition
## Lesson 1: Data Preprocessing and Model Diagnostics

## Pietro Vischia (Universidad de Oviedo and ICTEA), pietro.vischia@cern.ch

### A few global settings

- If you run on colab, set to `True` the relevant environmental variable.

This tutorial will assume you have a few GB free wherever you are running (locally on your machine, or in your google drive account)


In [ ]:
runOnColab = False

In [ ]:
if runOnColab:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd "/content/drive/MyDrive/"
    if not os.path.isdir("pv_data_science_schoool"): 
        %git clone https://github.com/vischia/pv_data_science_school.git
    %cd pv_data_science_school
#!pwd
#!ls
##!pip install livelossplot shap
#%pip install shap torchinfo livelossplot

In [ ]:
import matplotlib
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (8, 6)
matplotlib.rcParams['axes.labelsize'] = 14

import matplotlib.pyplot as plt
import glob
import os
#import re
#import math
#import socket
#import json
#import pickle
#import gzip
#import copy
#import array
import numpy as np
np.random.seed(42)
#import numpy.lib.recfunctions as recfunc

#from scipy.optimize import newton
#from scipy.stats import norm

import uproot

import datetime
from timeit import default_timer as timer

import sklearn
#from sklearn.model_selection import train_test_split
#from sklearn.utils import shuffle
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
#from sklearn.tree import export_graphviz
#from sklearn.inspection import permutation_importance
#try:
#    # See #1137: this allows compatibility for scikit-learn >= 0.24
#    from sklearn.utils import safe_indexing
#except ImportError:
#    from sklearn.utils import _safe_indexing
    
#from livelossplot import PlotLossesKeras

#from keras.losses import mean_squared_error, binary_crossentropy
#from keras.models import Sequential, Model
#from keras.layers import Input, Dense, Dropout, Concatenate
#from keras.layers import Lambda, Activation
#from keras.optimizers import SGD
#from tensorflow.keras.optimizers.legacy import Adam # for macos
## from keras.optimizers import Adam # for non-macos
#from keras.regularizers import l2
#from tensorflow.keras.layers import BatchNormalization
#from tensorflow.keras.layers import LeakyReLU

#import keras
#from keras import backend as K
#import tensorflow as tf
import pandas as pd

# The data set

We will use simulated events corresponding to three physics processes.

- ttH production
- ttW production
- Drell-Yan ($pp\to Z/\gamma^*$+jets) production

We will select the multilepton final state, which is a challenging final state with a rich structure and nontrivial background separation.

<img src="figs/2lss.png" alt="ttH multilepton 2lss" style="width:40%;"/>


In [ ]:
# This line ownloads the data only if you haven't done so yet

if not os.path.isfile("data/signal_blind20.root"): 
    !mkdir data; cd data/; wget https://www.hep.uniovi.es/vischia/lisbon_ml_school/lisbon_ml_school_tth.tar.gz; tar xzvf lisbon_ml_school_tth.tar.gz; rm lisbon_ml_school_tth.tar.gz; cd -;
else:
    print("Data were already donwloaded, I am not downloading them again.")

The data set is made of three files, one per background (`background_1` is ttW, `background_2`is Drell-Yan.
The ttH signal events file corresponds to only a percentage of the full data set: the rest is used in the data challenge (for schools where I offer a data challenge).

In [ ]:
import uproot

sig = uproot.open('data/signal_blind20.root')['Friends'].arrays(library="pd")
bk1 = uproot.open('data/background_1.root')['Friends'].arrays(library="pd")
bk2 = uproot.open('data/background_2.root')['Friends'].arrays(library="pd")


## Data inspection

The first thing you need to do when building a machine learning model is to forget about the model, and **just look at the data**


Let's start by looking at which features and labels are available in these files.


In [ ]:
sig.columns

Most of the variables are input features, corresponding to detector measurements of the properties of the reconstructed decay products.

There are three special variables, though:

- `Hreco_evt_tag`: this feature has values in ${0,1}$, where $1$ flags the event as signal event, and $0$ flags the event as background event;
- `Hreco_HTXS_Higgs_pt`: this feature contains the true generate Higgs boson transverse momentum at generator level (used for regression);
- `Hreco_HTXS_Higgs_y`: this feature contains the true generated Higgs boson rapidity (not pseudorapidity) at generator level (used for regression).

You'll see down below that we will have of course to ignore some of all of these three features (they are not input features).

You can also print a few lines from the dataset

In [ ]:
sig.head()

## Plotting histograms of some observables using matplotlib

(see also examples on [matplotlib](https://matplotlib.org/3.5.3/api/_as_gen/matplotlib.pyplot.html) website)

In [ ]:
plt.figure()
plt.hist(sig["Hreco_Lep0_pt"], bins=100)
plt.xlabel("Lepton 1 $p_T$")
plt.ylabel("Events")
plt.show()
plt.close()


We can also make scatter plots of one variable against the other: these are useful to assess the correlation between different features of the data.

We'll see a bit below a way of doing so in an elegant way for all the features of the data set.

In [ ]:
plt.figure()
plt.scatter(sig["Hreco_Lep0_pt"], sig["Hreco_Lep1_pt"])
plt.xlabel("Lepton0 pT")
plt.ylabel("Lepton1 pT")
plt.show()
plt.close()

We can also plot another variable, the transverse momentum of the third lepton

In [ ]:
plt.figure()
plt.hist(sig["Hreco_Lep2_pt"], bins=100)
plt.xlabel("Lepton0 pT")
plt.ylabel("Events")
plt.show()
plt.close()

What is going on in the last plot?


.


.


.


.


.


.


.


.


.

The events preselected in these files contain events that have **at least two** leptons with same electrical charge. We therefore have in the data set a mix of events with two same-sign leptons and three leptons (out of which necessarily two will have the same sign).

You have three choices:

1. train our algorithm on `2lss` events, by selecting only events where the third lepton transverse momentum **is not set**: `dataframe=dataframe[dataframe['Hreco_Lep2_pt']==-99]`
2. train our algorithm on `3l` events, by selecting only events where the third lepton transverse momentum **is set**: `dataframe=dataframe[dataframe['Hreco_Lep2_pt']>-99]`
3. train our algorithm on `2lss+3l` events. The default value of $-99$ in this case will act as a semi-categorical variable

Let's pick option (1).

We will first create a label for signal or background events (we could also use the `evt_tag` variable), then join all datasets together, then filter events to keep only those corresponding to 2lss events, and finally we will drop all the features corresponding to the third lepton, plus those corresponding to regression targets.


In [ ]:
# Create a new column 'label' and set its value to 1 or 0 for all rows (=events)
sig['label'] = 1 
bk1['label'] = 0
bk2['label'] = 0

# Merge the two backgrounds into one dataframe
bkg = pd.concat([bk1, bk2])

print(f"bkg1 shape {bk1.shape}")
print(f"bkg2 shape {bk2.shape}")
print(f"bkg1+bkg2 shape {bkg.shape}")

# Merge the signal and background into one dataframe
print(f" Signal shape {sig.shape}")
print(f" Bkg shape {bkg.shape}")

data = pd.concat([sig,bkg])

print(f" Data shape {data.shape}")
print(data.columns)

# Filter data
data=data[data['Hreco_Lep2_pt']==-99]
# Drop unneeded features
data = data.drop(["index","Hreco_Lep2_pt", "Hreco_Lep2_eta", "Hreco_Lep2_phi", "Hreco_Lep2_mass", 
                  "Hreco_evt_tag","Hreco_HTXS_Higgs_pt", "Hreco_HTXS_Higgs_y"], axis=1 )


print(f" Data shape {data.shape}")
print(data.columns)
print(f"In this dataframe we finally have {data[data['label']==1].shape[0]} signal and {data[data['label']==0].shape[0]} background events")
print("Sanity check: look for NAN numbers in any of the rows or columns")
print(data.isna().any())


This data set is still ordered, ie. all signal events are before the background events. Proper training requires a shuffled data set instead!

We could also do that (and in fact we will) when randomly separating our dataset in training and test events, but it doesn't hurt to do it from the very beginning, to avoid forgetting it.

We will also separate features and labels from each other, and check for corrupted values.


In [ ]:
data = data.sample(frac=1).reset_index(drop=True)
data.head(10)


print("There are NaN-filled elements:", data.isna().any().any())

X = data.drop(["label"], axis=1)
y = data["label"]

print(f"data shape {data.shape}")
print(f"input feature shape {X.shape}")
print(f"label (=target) shape {y.shape}")

We can inspect all the pairwise correlations between features, for signal and background separately, as well as one-dimensional densities of individual features, by doing a `pairplot` using the useful library `seaborn`.

In [ ]:
import seaborn as sns
sns.set()
cols_to_plot = [
    'Hreco_Lep1_pt',
    'Hreco_HadTop_pt',
    'Hreco_All5_Jets_pt',
    'Hreco_More5_Jets_pt',
    'Hreco_Jets_plus_Lep_pt',
    'label'
]
pp=sns.pairplot(data=data.sample(1000)[cols_to_plot], hue='label', diag_kws={'bw_method': 0.2}) # Scatter plot
pp.map_lower(sns.kdeplot, levels=4, color=".2") # Contours

Exercises:
- what happens if you plot the full list of variables (`cols_to_plot = data.columns`) from the command above?
- what happens if you omit `.sample(100)` from the command above?


### Split the data set into training and test set

When we train a machine learning algorithm, we are trying to solve an interpolation problem (*find the function of the input features that provides the best approximation of the true function*) by also requiring that the solution generalizes sufficiently well (*the interpolating function must also predict correctly the value of the true function for new, unseen data*).


When we have a labelled dataset, we will therefore split it into: a *training set*, which we will use to train the machine learning algorithm; a *test set*, which we will use to evaluate the performance of the algorithm for various realizations of the algorithm (e.g. tuning hyperparameters); and an *application set*, which are the data we are really interested in studying in the end.

For many applications, when the amount of hyperparameters tuning is moderate, application set and test set can be collapsed into a single set (usually called *test set*). This is what we will do in this tutorial.

![Blah](figs/trainingNetwork.png)

(Image: P. Vischia, [doi:10.5281/zenodo.6373442](https://doi.org/10.5281/zenodo.6373442))

In [ ]:
X_train_orig, X_test_orig, y_train_orig, y_test_orig = sklearn.model_selection.train_test_split(X, y, test_size=0.33, random_state=42)

print(f"We have {len(X_train_orig)} training samples with {sum(y_train_orig)} signal and {sum(1-y_train_orig)} background events")
print(f"We have {len(X_test_orig)} testing samples with {sum(y_test_orig)} signal and {sum(1-y_test_orig)} background events")


## Correlation matrices

For classification problems, another important thing is to take a look at the correlations between all the variables, in events belonging to each class separately.

Looking at the correlation between features can highlight pairs that are strongly correlated with each other, and one could decide to omit them since they do not add further information when the correlations are very high.

We look at the correlation for each class because we are very interested in pairs of features that have different correlation in one class or the other (in our example, signal or background).

Now we will choose a simple criterion, for instance the value of one of the features that characterize the houses, and use it to decide if the house is in New York (we want to predict 0) or in San Francisco (we want to predict 1)


In [ ]:
def corrmatrix(corr, label):
    plt.figure()
    ax = sns.heatmap(
        corr, 
        vmin=-1., vmax=1., center=0.,
        cmap=sns.diverging_palette(20., 220., n=200, as_cmap=True),
        square=True
    )
    ax.set_xticklabels(
        ax.get_xticklabels(),
        rotation=45,
        horizontalalignment='right'
    );

    ax.set_title('Correlation matrix for %s events' % label)

    plt.show()
    plt.close()


corrmatrix(X_train_orig[y_train_orig==1.].corr(), 'signal')
corrmatrix(X_train_orig[y_train_orig==0.].corr(), 'background')


What we have plotted is the Pearson correlation coefficient, which leads to an important limitation of this diagnostic tool.

The Pearson correlation coefficient captures only **linear** correlation between variables, and is blind to many nonlinear correlations that there may be. Don't trust the above matrices blindly.


![Figure from BDN2010](figs/corrcov.png)

(figure from C. Delaere slides at the 2010 BND school)

## Preprocess the data

Before training a first ML-based classifier we need to think about if any preprocessing of the data is required. Many ML algorithms are based on gradient minimization techniques that can fail if the inputs have numbers that widely-vary in magnitude. For example, the $p_T$ of a jet might range from 20 to 2000 GeV, covering several orders of magnitude, and can prevent the convergence of a minimization algorithm. In the case of $p_T$ a typical manual preprocessing could be to use the logarithm as input instead, ie. $\log(p_T/1~\textrm{GeV})$, which in particular moves.

In this data set, the features have already been scaled such that their range is around unity. Sometimes this happens naturally, but in this case several variables come directly from particle physics and represent momenta of particles produced in high-energy interaction: when expressed in GeV, these variables will most certainly **not** be in a range close to unity.

Common choices to preprocess input features automatically are minmax scaling or normalization.

#### Minmax\n",
Compress the range linearly:
$$X_{scaled} = \\\\frac{X-X_{min}}{X_{max}-X_{min}}$$
A drawback is that this results in an artificially smaller variance (the range is compressed linearly), which can deform the effect of outliers.

#### Standardization
Compress the range and the shape:
$$X_{normalized} = \\\\frac{X - \\\\mu}{\\\\sigma}$$
where $\\\\mu$ is the mean of the feature values and $\\\\sigma$ is the standard deviation.

#### Which one?\n",
- Typically one would use minmax scaling when your features are remarkably nongaussian and your ML algorithm of choice doesn't require Gaussian inputs. The price is that it affects outliers.
- Typically one would use normalization when the features are approximately Gaussian or when your ML algorithm of choice requires Gaussian inputs. However, it also results in numbers close to 1 (minimization algorithms and gradient descend love numbers that are not too large or too small), so it can be used for any algorithm: the good news is that it doesn't affect outliers.

You can also apply PCA, as we have discussed in the lectures. The code below imports the most common scalers, for you to play with. For source identification e.g. in the processing of periodic signals (sound waves, radio signals, etc), you can also use ICA (Indipendent Component Analysis), which is rather powerful.

For now, let's use the standard scaler for the input features of the dense NN. You are encouraged to try out the others!

We will also store the original train and test structures, because for the convolutional network we will preprocess them differently.

We will also store the original train and test structures, so that you can try out various preprocessing techniques and play at the same time with many of them.


In [ ]:
X_train_conv = X_train_orig.copy()
y_train_conv = y_train_orig.copy()
X_test_conv = X_test_orig.copy()
y_test_conv = y_test_orig.copy()

X_train = X_train_orig.copy()
y_train = y_train_orig.copy()
X_test = X_test_orig.copy()
y_test = y_test_orig.copy()


from sklearn.preprocessing import (
    MaxAbsScaler, # maxAbs
    MinMaxScaler, # MinMax
    Normalizer, # Normalization (equal integral)
    StandardScaler# standard scaling
)
from sklearn.decomposition import PCA

# Scale the input features and the target variable
for column in X_train.columns:
    scaler = StandardScaler().fit(X_train.filter([column], axis=1))
    X_train[column] = scaler.transform(X_train.filter([column], axis=1))
    X_test[column] = scaler.transform(X_test.filter([column], axis=1))


You could also use the basic syntax recommended by the documentation, as follows, but then you would be standardizing all the features to exacly the same mean. This may work for some data sets, but for this specific one it does not (it actually significantly worsens the performance---you can try ;) ).

```
scaler = StandardScaler().fit(X_train[X_train.columns])
X_train[X_train.columns] = scaler.transform(X_train[X_train.columns])
X_test[X_test.columns] = scaler.transform(X_test[X_test.columns])
```

### Build a simple tree-based classifier

The act of selecting regions of a data set by "cutting" (imposing thresholds) on some of the features is very natural for the particle physicist, so let's start by training a tree-based classifier.

Decision trees are precisely that:

<img src="figs/bdt_en_edit.png" alt="bdtexample" style="width:80%;"/>

(image from [r2d3.us](http://www.r2d3.us/visual-intro-to-machine-learning-part-1/))

Decision trees depend however on the random order of imposing cuts on the data set. To reduce the dependence on the starting point and the ordering of the selection process, we will use an *ensemble* of decision trees, which will cut at random the data set, and we will *pool* the classification answers from all the trees via e.g. a majority vote.

Not all the trees resulting from the random cuts will make sense, so we will use a procedure called *boosting*, which consists in weighting each random tree based on its own performance. The weights will be used to decide how to generate the next trees.


<img src="figs/boosting.png" alt="boosting" style="width:80%;"/>


In [ ]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

learning_rate = 0.1


bdt_ada_orig = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=3, criterion='log_loss'), n_estimators=100, learning_rate=learning_rate, random_state=42)
bdt_grad_orig = GradientBoostingClassifier(n_estimators=100, learning_rate=learning_rate, max_depth=3, random_state=42, verbose=1)

bdt_ada = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=3, criterion='log_loss'), n_estimators=100, learning_rate=learning_rate, random_state=42)
bdt_grad = GradientBoostingClassifier(n_estimators=100, learning_rate=learning_rate, max_depth=3, random_state=42, verbose=1)

In [ ]:
# If it takes too much to run, try downsampling:
Ntrain=10000
Ntest=2000
X_train = X_train[:Ntrain]
y_train = y_train[:Ntrain]
X_test = X_test[:Ntest]
y_test = y_test[:Ntest]

X_train_orig = X_train_orig[:Ntrain]
X_test_orig = X_test_orig[:Ntest]

print("Training with adaptive boost")
fitted_bdt_ada=bdt_ada.fit(X_train, y_train)
fitted_bdt_ada_orig=bdt_ada_orig.fit(X_train_orig, y_train)

print("Training with gradient boost")
fitted_bdt_grad=bdt_grad.fit(X_train, y_train)
fitted_bdt_grad_orig=bdt_grad_orig.fit(X_train_orig, y_train)


### Estimate the performance of the classifier

A simple performance estimate for the classifier is the mean accuracy on a certain data set.
Let's print it out for the training set and for the test set.

Let's also print some metric as a function of the training iteration. Since this is a classification problem, we will use the accuracy on the test sample.

Accuracy is defined as $\frac{\text{correct predictions}}{\text{all predictions}}$


In [ ]:
print('Adaptive boost: train accuracy', fitted_bdt_ada.score(X_train, y_train),', test accuracy', fitted_bdt_ada.score(X_test, y_test))
print('Gradient boost: train accuracy', fitted_bdt_grad.score(X_train, y_train),', test accuracy', fitted_bdt_grad.score(X_test, y_test))

print('Adaptive boost (non-scaled data): train accuracy', fitted_bdt_ada_orig.score(X_train_orig, y_train),', test accuracy', fitted_bdt_ada_orig.score(X_test_orig, y_test))
print('Gradient boost (non-scaled data): train accuracy', fitted_bdt_grad_orig.score(X_train_orig, y_train),', test accuracy', fitted_bdt_grad_orig.score(X_test_orig, y_test))


def get_loss_vs_iteration(x_features, y_labels, classifier):
    return np.array(list(map(
        lambda score: sklearn.metrics.log_loss(y_labels,score), classifier.staged_predict_proba(x_features)
    )))

train_loss_bdt_ada = get_loss_vs_iteration(X_train, y_train, fitted_bdt_ada)
test_loss_bdt_ada = get_loss_vs_iteration(X_test, y_test, fitted_bdt_ada)


train_loss_bdt_grad = get_loss_vs_iteration(X_train, y_train, fitted_bdt_grad)
test_loss_bdt_grad = get_loss_vs_iteration(X_test, y_test, fitted_bdt_grad)


plt.figure()
plt.plot(np.arange(len(train_loss_bdt_ada)),train_loss_bdt_ada,label="Ada boost (train)",color='royalblue',linestyle='-')
plt.plot(np.arange(len(test_loss_bdt_ada)),test_loss_bdt_ada,label="Ada boost (test)",color='royalblue',linestyle='--')

plt.plot(np.arange(len(train_loss_bdt_grad)),train_loss_bdt_grad,label="Gradient boost (train)",color='orange',linestyle='-')
plt.plot(np.arange(len(test_loss_bdt_grad)),test_loss_bdt_grad,label="Gradient boost (test)",color='orange',linestyle='--')
         
plt.ylabel("Loss")
plt.xlabel("Iteration")
plt.legend()
plt.show()
plt.close()

We can observe two things:

The performance of gradient-based boosting is better than of adaptive boosting, which is normally expected.

For adaptive boosting, after about 20 iterations the performance on both the training and test data sets starts degrading: it's clear that the training should stop at about 20 iterations.

For gradient boosting, the performance on the train set begins to be remarkably lower than that on the training set after a few iteration, and the performance on the training set keeps improving. All of this is an indication that our algorithm is able to separate the two classes (signal and background) very effectively, but generalizes somehow poorly to data not seen during the training. Remember the definition of overtraining given during the lectures.

To study the generalization properties of machine learning algorithms we will switch to neural networks, where we can control in an intuitive way the complexity of the network.

For the moment, however, let's look at some additional ways of exploring the algorithm's performance.

Exercise:
- What happens if you change the learning rate to `1.0`? What happens if you change it to `0.01`? Do you need to tweak the amount of iterations?

Before, though, let's compare the training of our scaled vs non-scaled classifiers:

In [ ]:

train_loss_bdt_ada_orig = get_loss_vs_iteration(X_train_orig, y_train, fitted_bdt_ada_orig)
test_loss_bdt_ada_orig = get_loss_vs_iteration(X_test_orig, y_test, fitted_bdt_ada_orig)


train_loss_bdt_grad_orig = get_loss_vs_iteration(X_train_orig, y_train, fitted_bdt_grad_orig)
test_loss_bdt_grad_orig = get_loss_vs_iteration(X_test_orig, y_test, fitted_bdt_grad_orig)


plt.figure()
plt.plot(np.arange(len(train_loss_bdt_ada)),train_loss_bdt_ada-train_loss_bdt_ada_orig,label="Ada boost (train)",color='royalblue',linestyle='-')
plt.plot(np.arange(len(test_loss_bdt_ada)),test_loss_bdt_ada-test_loss_bdt_ada_orig,label="Ada boost (test)",color='royalblue',linestyle='--')

plt.plot(np.arange(len(train_loss_bdt_grad)),train_loss_bdt_grad-train_loss_bdt_grad_orig,label="Gradient boost (train)",color='orange',linestyle='-')
plt.plot(np.arange(len(test_loss_bdt_grad)),test_loss_bdt_grad-test_loss_bdt_grad_orig,label="Gradient boost (test)",color='orange',linestyle='--')
         
plt.ylabel("loss_scaled-loss_unscaled")
plt.xlabel("Iteration")
plt.legend()
plt.show()
plt.close()

#### What is going on???

Boosted decision trees are an ensemble method that consists in building many decision trees, i.e. classifiers based on thresholds ("cuts", in particle physics) on the features.

Scalers modifies the values of each feature but **they preserve the ordering**, and therefore scaling or not scaling doesn't change in any way the results of the algorithm.

You will see in the next tutorial that scaling is, on the contrary, very important for training neural networks.

#### The classifier output

Let's take a look at the resulting classfiers, for the two classes and the two algorithms. This is always done exclusively on the test data set.

The `decision_function(input, k)` gives a score that, for binary classification ($k==1$) is closer to -1 if the event is classified as background-like and closer to +1 if the event is classified as signal-like.

In [ ]:
plt.figure()
plt.hist(fitted_bdt_ada.decision_function(X_test[y_test==0]), bins=20, alpha=0.7, label="Score for background events")
plt.hist(fitted_bdt_ada.decision_function(X_test[y_test==1]), bins=20, alpha=0.7, label="Score for signal events")
plt.title("Adaptive Boost")
plt.yscale("log")
plt.show()
plt.close()

plt.figure()
plt.hist(fitted_bdt_grad.decision_function(X_test[y_test==0]), bins=20, alpha=0.7, label="Score for background events")
plt.hist(fitted_bdt_grad.decision_function(X_test[y_test==1]), bins=20, alpha=0.7, label="Score for signal events")
plt.title("Gradient Boost")
plt.yscale("log")
plt.show()
plt.close()

How do we interpret these distributions?

#### ROC curve

As we have seen in the lecture, the Receiver Operating Characteristic (ROC) curve is a handy performance plot for the training of a classifier

In [ ]:
def plot_rocs(scores_and_names, y):
    pack=[] 
    for s, n in scores_and_names: 
        fpr, tpr, thresholds = roc_curve(y.ravel(), s)
        pack.append([n, fpr,tpr,thresholds])

    plt.figure()
    lw=2
    for n, fpr, tpr, thresholds in pack:
        plt.plot(fpr, tpr, lw=lw, label="%s (AUC = %0.2f)" % (n, auc(fpr, tpr))) 

    plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver Operating Characteristic curve")
    plt.legend(loc="lower right")
    plt.show()


Let's now plot the ROC curve for our two classifiers. To do so, we have first to convert the 

In [ ]:
plot_rocs([ [fitted_bdt_ada.decision_function(X_test), 'AdaBoost'],
            [fitted_bdt_grad.decision_function(X_test), 'GradBoost']],
          y_test)

The ROC curve only reports the TPR and FRP for all possible thresholds of the classifier.
Once we decide a threshold (conventionally, 0.5), we can also plot the confusion matrix for that choice.

In the case of ensemble classifiers such as boosted decision trees, `predict` gives the predicted class of an input sample as the weighted mean prediction of the classifiers in the ensemble.

In [ ]:
def print_and_plot_confusion_matrix(clf, X, y, label=""):
    cm = confusion_matrix(y, clf.predict(X), labels=clf.classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
    plt.figure()
    disp.plot()
    plt.title(label)
    plt.show()
    plt.close()
    print("Confusion matrix:")
    print(cm)

print_and_plot_confusion_matrix(fitted_bdt_ada, X_test, y_test, "Adaptive Boost")
print_and_plot_confusion_matrix(fitted_bdt_grad, X_test, y_test, "Gradient Boost")


The classifiers are pretty bad, right?

Exercise:
- can you find ways of improving them? In the next exercises we will train a neural network to solve the same problem, but for now you can try:
    - tweak parameters of the classifiers (learning rate, iterations, etc)
    - tweak the data preprocessing

## Inspection of the model

We can also inspect in detail the model structure and its correlation with a dataset.

In [ ]:
import sklearn.tree

plt.figure(figsize=[25,18])
sklearn.tree.plot_tree(fitted_bdt_grad.estimators_[42, 0],feature_names=data.columns, fontsize=14)
plt.show()
plt.close()

In [ ]:
###Alternative method:# Pick one of the trees (maybe modify to pick the tree with largest weight or something like that)
#sub_tree_42 = fitted_bdt_grad.estimators_[42, 0]
## Visualization
## Install graphviz: https://www.graphviz.org/download/
#from pydotplus import graph_from_dot_data
#from IPython.display import Image
#dot_data = export_graphviz(
#    sub_tree_42,
#    out_file=None, filled=True, rounded=True,
#    special_characters=True,
#    proportion=False, impurity=False, # enable them if you want
#)
#graph = graph_from_dot_data(dot_data)
#Image(graph.create_png())

## Explainability

A good question to pose yourself is whether the features you have chosen for your data are meaningful variables (i.e. if they are actually relevant to your classifier). Another good question is which features drive the prediction for a given event or set of events.

All these questions can be answered by using different concepts:

- **Permutation importance**: the decrease in a model score when a single feature value is randomly shuffled (scikit-learn docs) (akin to impacts for profile likelihood fits)
Shapley values: based on game theory (see other contribution)
Correlation-based: e.g. parallel coordinates in TMVA: look where each variable is mapped
to/correlated with

- **Perturbational approach**: perturbing the value of a feature and looking at the change of the prediction gives hints on how important the variable is for the method. This is at the basis of LIME (`pip install lime`). You can read more [here](https://www.oreilly.com/content/introduction-to-local-interpretable-model-agnostic-explanations-lime/)

- **Game-theoretical approach**: consider the prediction task as a game in game theory, and the features as players who bet via their values. The payout, as difference of prediction with respect to the true value, estimates how much a feature pushes the prediction away from the truth.

- **Visual approach**: parallel coordinates, which were implemented in ROOT TMVA, let you handily select a range in the prediction, and have a visual representation of which ranges of each feature is mapped into that region of the prediction.

![Parallel coordinates (reference in the figure)](figs/parcoord.png)

###### Permutation importance

The idea is: randomly shuffle one single feature value, then check how much does the prediction change. If the prediction decreases by a lot, then the value of the feature is crucial to the prediction.

Note: the importance is always **relative to a specific model**, it has no absolute validity. A feature that is deemed low-importance for a badly designed model may be deemed high-importance for a good model, and viceversa. Permutation importance scores don't "talk" across different models.

Also, if the model has a performance which is near-chance, then it is not strongly predictive, so the answers one may get from permutation importance scores are not really reliable.

In [ ]:
from sklearn.inspection import permutation_importance

scoring = ['r2', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error']

rs = permutation_importance(
    fitted_bdt_grad, X_test, y_test, n_repeats=30, random_state=0, scoring=scoring)

for metric in rs:
    print(f"{metric}")
    r = rs[metric]
    for i in r.importances_mean.argsort()[::-1]:
        if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
            print(f"    {X_train.columns[i]:<8}" # +1 to skip the label in the naming
                  f"{r.importances_mean[i]:.3f}"
                  f" +/- {r.importances_std[i]:.3f}")



##### Shapley values
(based on [this blog post](https://www.analyticsvidhya.com/blog/2019/11/shapley-value-machine-learning-interpretability-game-theory/))

Shapley values are a construct based on game theory.
The main idea behind Shapley values is to consider the prediction task for a single event as game played by the feature values of that event. The features collaborate together to play the game by betting. The value of the feature is the amount each feature bets on the prediction task. The **Shapley Value** for each feature is the payout of the game, and consists in the correct weight such that the sum of all Shapley values for the features is the difference between the predictions and the average value of the model. In other words, the Shapley value represents how much each variable pushes the prediction far from the expected value.

More concretely, the Shapley value for a feature A is computed as follows:

- Get all subsets of features that do not contain A
- Compute the effect of adding A to each of these subsets 
- Aggregate all the contributions (i.e. compute the marginal contribution of the feature over all the subsets)

In principle we should retrain the model for each of these subsets, but instead we (the `shap` package, actually) will just compute predictions by replacing the value of the feature with its own average value.



In [ ]:
import shap
shap.initjs()
explainer = shap.TreeExplainer(fitted_bdt_grad)
shap_values = explainer.shap_values(X_train)
i = 500
shap.force_plot(explainer.expected_value, shap_values[i], features=X_train.iloc[i], feature_names=X_train.columns)

Values in blue represent features that push the prediction towards negative values, values in red represent features that push the prediction towards positive values, *for the event number 4776*.

We naturally want a summary of Shapley values over all observations:

In [ ]:
shap.summary_plot(shap_values, features=X_train, feature_names=X_train.columns, use_log_scale=True)

Besides looking at the more important variables, you may also look at the less important, to **prune** them.

Pruning consists in dropping the least important variables and retraining your machine learning algorithm.
The idea behind it is that the variables dropped don't influence the prediction anyway, and retraining without them should give more or less the same performance but with a simpler model. Why would we do that? Well, for example inference may be time-sensitive, and simpler models are computationally **faster** to evaluate.

Exercise:
- Prune the least important variables and train new classifiers with less variables. Compare the performance, and time the execution using the python library `time`

### The end (for now)